In [10]:
# датасеты
from datasets import load_dataset
# время обработки задач
from tqdm.auto import tqdm
# пути в операционке (заменить на OS)
from pathlib import Path
import os
from pathlib import Path

In [67]:
# например, баскский датасет
dataset = load_dataset('oscar', 'unshuffled_deduplicated_eu') 

In [73]:
# проверили, что чего-то в нём действительно есть
dataset["train"][1]

{'id': 1,
 'text': 'Erabiltzaile bati goitizen bat (alias) erantsi diezaiokegu.Honek zure gmaileko mezuak jasotzeko edo bidaltzeko balioko du. Baina, postontzi bakarra da, ez da beste erabiltzaile edo kontu bat. Beraz, mezuak jatorrizko kontu (erabiltzaile) hortatik bidaliko dituzu eta bertara iritsiko dira.\nHau irakasten ditugun arloak bereizteko baliogarria izan daiteke. Horrela ikasgai bateko ikasleek bidaltzen dizkiguten mezuak, gmaileko iragazi bat ezarriz, sailkatu ditzakegu. Adibidez, gure oinarrizko helbidea josebe.anda@gureikastetxea.net bada, josebe.anda.matematika@gureikastetxea.net goitizena jar genezake, eta matematikako ikasleekin eposta harremanak goitizen horrekin eduki.\nIkastetxeko karguekin ez da oso komenigarria goitizenak jartzea, irakasle horrek kargua uztean goitizenera iritsi zaizkion mezu guztiak bere oinarrizko gmail kontuan geldituko baitira.'}

In [11]:
# сделали папку, чтобы хранить эту радость
eu_path = './data/text/oscar_eu/'
os.makedirs(eu_path, exist_ok=True)

In [13]:
text_data = []
file_count = 0
for sample in dataset["train"]:
    sample_text = sample['text'].replace('\n', '')
    text_data.append(sample_text) # каждый текст в датасете запоминаем
    
    if len(text_data) == 10_000: # каждые 10к текстов сохраняем в новый файлик
        with open(os.path.join(eu_path, f'text_{file_count}.txt'), 'w', encoding='utf-8') as file:
            file.write('\n'.join(text_data))
        text_data = []
        file_count += 1

# остатки тоже сохранили
if text_data:
    with open(os.path.join(eu_path, f'text_{file_count}.txt'), 'w', encoding='utf-8') as file:
        file.write('\n'.join(text_data))

In [66]:
# пути до текстов:
tokenization_files = [str(x) for x in Path('./data/text/oscar_eu').glob('**/*.txt')]
# tokenization_files

In [25]:
from tokenizers import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer()
tokenizer.train(files=tokenization_files[:100], vocab_size=30_522, min_frequency=2,
                special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

In [26]:
os.mkdir('./lil_eu_model')
tokenizer.save_model('lil_eu_model')

['lil_eu_model\\vocab.json', 'lil_eu_model\\merges.txt']

In [8]:
from transformers import RobertaTokenizerFast

Теперь обучим на полном датасете

In [7]:
tokenizer = ByteLevelBPETokenizer()
tokenizer.train(files=tokenization_files, vocab_size=50_000, min_frequency=2,
                special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])


NameError: name 'ByteLevelBPETokenizer' is not defined

In [209]:
tokenizer

Tokenizer(vocabulary_size=50000, model=ByteLevelBPE, add_prefix_space=False, lowercase=False, dropout=None, unicode_normalizer=None, continuing_subword_prefix=None, end_of_word_suffix=None, trim_offsets=False)

In [9]:
os.mkdir('./full_eu_0')
tokenizer.save_model('full_eu_0')

NameError: name 'os' is not defined

In [11]:
best_possible_tokenizer = RobertaTokenizerFast.from_pretrained('full_eu_0')

MLM

In [12]:
import torch
from copy import deepcopy

In [13]:
def mlm(tensor):
    # mask_arr = (torch.rand(tensor.shape) < 0.15) * (tensor != 0) * (tensor != 1) * (tensor != 2)
    rand = torch.rand(tensor.shape)
    # mask_arr = (rand < 0.15) * (tensor != 0) * (tensor != 1) * (tensor != 2)
    mask_arr = (rand < 0.15) * (tensor > 2)
    for i in range(tensor.shape[0]):
        selection = torch.flatten(mask_arr[i].nonzero()).tolist()
        tensor[i, selection] = 4
    return tensor

In [14]:
# tokenization_files # всё ещё все файлы
input_ids = []
mask = []
labels = []

In [17]:
for path in tqdm(tokenization_files):
    with open(path, 'r', encoding="utf-8") as file:
        lines = file.read().split('\n')
    sample = best_possible_tokenizer(lines, max_length=512, padding='max_length', truncation=True, return_tensors='pt') # truncation=true обрезать слишком длинные строки
    labels.append(sample.input_ids)
    mask.append(sample.attention_mask)
    input_ids.append(mlm(sample.input_ids.detach().clone())) # .detach().clone() — как deepcopy мы не хотим портить настоящий список с id, т. к. он нужен в labels

  0%|          | 0/27 [00:00<?, ?it/s]

In [20]:
input_ids

tensor([[    0,  2658, 31583,  ...,     1,     1,     1],
        [    0, 31903,  2135,  ...,     1,     1,     1],
        [    0,    37, 24309,  ...,     1,     1,     1],
        ...,
        [    0, 46864,     4,  ...,     1,     1,     1],
        [    0,     4,    30,  ...,     1,     1,     1],
        [    0, 23202,     4,  ...,     1,     1,     1]])

In [19]:
# можно было и сразу нормально записывать, нет?..
input_ids = torch.cat(input_ids)
mask = torch.cat(mask)
labels = torch.cat(labels)

TypeError: cat(): argument 'tensors' (position 1) must be tuple of Tensors, not Tensor

In [ ]:
# torch.save(labels, 'eu_labels.pt') # можно схранить

Датасет

In [21]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # store encodings internally
        self.encodings = encodings

    def __len__(self):
        # return the number of samples
        return self.encodings['input_ids'].shape[0]

    def __getitem__(self, i):
        # return dictionary of input_ids, attention_mask, and labels for index i
        return {key: tensor[i] for key, tensor in self.encodings.items()}

In [22]:
{'input_ids': input_ids, 'attention_mask': mask, 'labels': labels}

{'input_ids': tensor([[    0,  2658, 31583,  ...,     1,     1,     1],
         [    0, 31903,  2135,  ...,     1,     1,     1],
         [    0,    37, 24309,  ...,     1,     1,     1],
         ...,
         [    0, 46864,     4,  ...,     1,     1,     1],
         [    0,     4,    30,  ...,     1,     1,     1],
         [    0, 23202,     4,  ...,     1,     1,     1]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'labels': tensor([[    0,  2658, 31583,  ...,     1,     1,     1],
         [    0, 31903,  2135,  ...,     1,     1,     1],
         [    0,    37, 24309,  ...,     1,     1,     1],
         ...,
         [    0, 46864,  6160,  ...,     1,     1,     1],
         [    0, 23022,    30,  ...,     1,     1,     1],
         [    0, 23202,  2019,  ...,     1,     1,    

In [23]:
dataset = Dataset({'input_ids': input_ids, 'attention_mask': mask, 'labels': labels})

In [24]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True) # ???

In [25]:
loader.dataset

In [219]:
# dir(loader)

In [26]:
from transformers import RobertaConfig
from transformers import RobertaForMaskedLM

config = RobertaConfig(
    vocab_size=50_000,  # we align this to the tokenizer vocab_size
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1
)
model = RobertaForMaskedLM(config)

In [27]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): La

In [225]:
# model

In [28]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=1e-4)

C:\Users\m092\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [29]:
from tqdm import tqdm
epochs = 2

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        print(attention_mask)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        loss = outputs.loss
        loss.backward()
        optim.step()
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|                                                                                        | 0/16658 [00:00<?, ?it/s]

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


Epoch 0:   0%|                                                           | 1/16658 [00:19<91:04:49, 19.68s/it, loss=11]

tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


Epoch 0:   0%|                                                         | 2/16658 [00:38<88:21:14, 19.10s/it, loss=10.7]

...

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


Epoch 0:   9%|████▍                                              | 1451/16658 [7:29:56<78:20:05, 18.54s/it, loss=0.471]

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])


Epoch 0:   9%|████▍                                              | 1451/16658 [7:30:15<78:38:50, 18.62s/it, loss=0.471]


KeyboardInterrupt: 

In [48]:
model.save_pretrained('./full_eu')

Что вышло

In [31]:
from transformers import pipeline

In [49]:
fill = pipeline('fill-mask', model='full_eu', tokenizer='full_eu_0')

In [ ]:
jo dut katua
 

In [74]:
fill(f'dizkiguten {fill.tokenizer.mask_token}, ')

[{'score': 0.09843968600034714,
  'token': 16,
  'token_str': ',',
  'sequence': 'dizkiguten,, '},
 {'score': 0.03133409470319748,
  'token': 279,
  'token_str': ' eta',
  'sequence': 'dizkiguten eta, '},
 {'score': 0.031198779121041298,
  'token': 18,
  'token_str': '.',
  'sequence': 'dizkiguten., '},
 {'score': 0.012987540103495121,
  'token': 298,
  'token_str': ' da',
  'sequence': 'dizkiguten da, '},
 {'score': 0.011199723929166794,
  'token': 17,
  'token_str': '-',
  'sequence': 'dizkiguten-, '}]

In [39]:
fill.tokenizer.mask_token

'<mask>'